# Notebook 01: Introduction to RAG - Semantic Search Basics

## 🎯 Your Mission

You're an IT support engineer learning to use semantic search with RAG. Your job today: query a pre-indexed vector database to find IT tickets using semantic similarity - matching meaning, not just keywords.

**Why this matters:** This same RAG (Retrieval-Augmented Generation) approach is how you could build search systems that understand meaning, not just keywords - enabling faster incident resolution by finding similar past problems and their solutions.

**📋 Prerequisites:** 
- Complete `00_data_ingestion.ipynb` first (it creates both vector stores)
- The single-field vector store will be automatically found by name

---

## ⚡ Quick Win: See Semantic Search in Action

**The Problem:** Traditional keyword search only finds exact matches. If you search for "application crashes", it won't find tickets about "software failures" or "system errors" - even though they're the same problem!

**The Solution:** Semantic search understands meaning. It knows that "application crashes", "software failures", and "system errors" are similar concepts.

**Try this:** After we build the system, search for "application crashes" and watch it find tickets with different wording but the same meaning!

**💡 This is the power of RAG - matching by meaning, not just keywords.**

---

## 🎯 What You'll Learn

By the end of this notebook, you will:
- ✅ Build a simple RAG system that indexes and searches IT tickets
- ✅ Understand how semantic search works (matching meaning, not keywords)
- ✅ See the difference between keyword search and semantic search
- ✅ Use RAG to answer questions using retrieved ticket context

**Time:** ~15-20 minutes

---

## 📋 The Journey

We'll query the RAG system step by step:

1. **Explore the Data** - See what tickets look like
2. **Understand RAG** - Learn how semantic search works
3. **Set Up LlamaStack** - Connect to our RAG platform
4. **Load Vector Store** - Connect to the pre-indexed vector database
5. **Query & Search** - Find tickets by meaning, not keywords

---

### Step 1: Explore the Dataset

**What we're doing:** Loading IT call center tickets and examining their structure.

**Why:** We need to understand what data we're working with before we can index it for semantic search.


In [ ]:
# Import required libraries
import pandas as pd
from pathlib import Path
import uuid
from llama_stack_client import RAGDocument

# Load the CSV file from the data directory
data_dir = Path("../data")
file_path = data_dir / "synthetic-it-call-center-tickets-sample.csv"

print("🔄 Loading IT call center tickets dataset...")
df = pd.read_csv(file_path)

print(f"✅ Loaded {len(df)} tickets")
print(f"📋 Dataset shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\n🔍 Let's examine the dataset:")
print("=" * 60)
df.head()

**What we see:** Each ticket has a `short_description` field that describes the problem. This is what we'll index for semantic search.

**💡 Key insight:** Traditional keyword search would only find exact matches. RAG uses semantic similarity - it understands that "application crashes" and "software failures" mean similar things!

Let's examine some example tickets to understand what we're working with:

In [ ]:
# Show dataset structure and example tickets
print("📊 Dataset Structure:")
print("=" * 60)
print(f"\nColumns: {list(df.columns)}")
print(f"\n📝 Key Field for Simple RAG:")
print(f"   - short_description: Problem summary (this is what we'll index)")

# Show example tickets with more detail
print("\n📋 Example Tickets:")
print("=" * 60)
if len(df) > 0:
    for i in range(min(5, len(df))):
        example = df.iloc[i]
        desc = str(example.get('short_description', 'N/A'))
        print(f"\n🎫 Ticket #{example.get('number', 'N/A')}")
        print(f"   Description: {desc}")
        if len(desc) > 100:
            print(f"   (Full description: {desc[:150]}...)")
    
    print(f"\n💡 Notice how each ticket describes a problem differently!")
    print(f"   Traditional search: Would only find exact word matches")
    print(f"   Semantic search: Finds tickets with similar meaning, even different words")

---

### Step 2: Understand How RAG Works

**What we're learning:** How semantic search differs from traditional keyword search.

**Why:** Understanding the concept helps you see why RAG is powerful for IT operations.

#### 🔍 Keyword Search (Traditional)

```
Search: "application crashes"
Results: Only tickets with exact words "application" AND "crashes"
❌ Misses: "software failures", "system errors", "app stops working"
```

#### 🧠 Semantic Search (RAG)

```
Search: "application crashes"  
Results: Tickets with similar meaning:
  ✅ "application crashes"
  ✅ "software failures"  
  ✅ "system errors"
  ✅ "app stops working"
  ✅ "program terminates unexpectedly"
```

**How it works:**
1. **Indexing:** Convert ticket descriptions into "embeddings" (vectors that capture meaning)
2. **Querying:** Convert your search query into the same type of embedding
3. **Matching:** Find tickets whose embeddings are similar to your query embedding
4. **Retrieval:** Return the most semantically similar tickets

**💡 Think of it like:** Instead of matching words, we're matching meanings. The system understands that "crash" and "failure" mean similar things in context.

---


### Step 3: Set Up LlamaStack Client

**What we're doing:** Connecting to LlamaStack - our RAG platform that handles:
- Vector database (stores ticket embeddings)
- Embedding generation (converts text to vectors)
- Semantic search (finds similar tickets)

**Why:** LlamaStack provides all the RAG infrastructure we need, so we can focus on building the search system.

**What happened:** We explored the dataset and understand how semantic search works. Now let's connect to LlamaStack to start building our RAG system.

**What's next:** We'll initialize the LlamaStack client and verify the connection.

**Ready to connect?** Let's initialize the LlamaStack client:

In [ ]:
# Import required libraries for LlamaStack
import os
import sys
from pathlib import Path
from llama_stack_client import LlamaStackClient
from termcolor import cprint

# Add root src directory to path to import shared config
root_dir = Path("../..").resolve()
sys.path.insert(0, str(root_dir / "src"))

# Import centralized configuration
from config import LLAMA_STACK_URL, MODEL, CONFIG

# Configuration values (automatically detected based on environment)
llamastack_url = LLAMA_STACK_URL
model = MODEL

if not llamastack_url:
    raise ValueError(
        "LLAMA_STACK_URL is not configured!\n"
        "Please run: ./scripts/setup-env.sh\n"
        "Or set LLAMA_STACK_URL environment variable:\n"
        "  export LLAMA_STACK_URL='https://llamastack-route-my-first-model.apps.ocp.example.com'"
    )

print("🔄 Connecting to LlamaStack...")
print("=" * 60)
print(f"📡 LlamaStack URL: {llamastack_url}")
print(f"🤖 Model: {model}")
print(f"📍 Environment: {'Inside OpenShift cluster' if CONFIG['inside_cluster'] else 'Outside OpenShift cluster'}")
print(f"📦 Namespace: {CONFIG['namespace']}")

# Initialize LlamaStack client
client = LlamaStackClient(base_url=llamastack_url)

# Verify connection
try:
    models = client.models.list()
    model_count = len(models.data) if hasattr(models, 'data') else len(models)
    print(f"\n✅ Connected to LlamaStack")
    print(f"   Available models: {model_count}")
except Exception as e:
    print(f"\n❌ Cannot connect to LlamaStack: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Check if route exists: oc get route llamastack-route -n my-first-model")
    print("   2. Run setup script: ./scripts/setup-env.sh")
    print("   3. Or set LLAMA_STACK_URL manually in .env file")
    raise

# Configure inference parameters
temperature = float(os.getenv("TEMPERATURE", 0.0))
max_tokens = int(os.getenv("MAX_TOKENS", 4096))
stream_env = os.getenv("STREAM", "True")
stream = (stream_env != "False")

print(f"\n⚙️  Inference Parameters:")
print(f"   Model: {model}")
print(f"   Temperature: {temperature}")
print(f"   Max Tokens: {max_tokens}")
print(f"   Stream: {stream}")

**What happened:** We connected to LlamaStack successfully! ✅

**What's next:** Now we'll create a vector store (where ticket embeddings will be stored) and then index our tickets.

---

### Step 4: Load Vector Store

**What we're doing:** Connecting to the pre-indexed vector store created in `00_data_ingestion.ipynb`.

**Why:** The vector store already contains indexed tickets. We just need to connect to it to start querying.

**📋 Prerequisites:** 
- You must have completed `00_data_ingestion.ipynb` (it creates both vector stores)
- The single-field vector store will be automatically found by name

**💡 How it works:**
- The ingestion notebook creates a vector store named `single-field-rag-tickets`
- This notebook will automatically find it by name
- Or you can manually set `VECTOR_STORE_ID_SINGLE_FIELD` if needed

In [ ]:
# Load vector store ID
# Option 1: Use the vector store ID from the ingestion notebook (if you ran it in the same session)
# Option 2: Manually set it here if you have a saved vector store ID

print("\n🔄 Step 4: Loading vector store...")
print("=" * 60)

# Try to use VECTOR_STORE_ID_SINGLE_FIELD from ingestion notebook, or find by name
try:
    # Check if VECTOR_STORE_ID_SINGLE_FIELD exists from ingestion notebook
    if 'VECTOR_STORE_ID_SINGLE_FIELD' in globals():
        vector_store_id = VECTOR_STORE_ID_SINGLE_FIELD
        VECTOR_STORE_ID = VECTOR_STORE_ID_SINGLE_FIELD  # For backward compatibility
        print(f"✅ Using vector store ID from ingestion notebook")
    else:
        raise NameError("VECTOR_STORE_ID_SINGLE_FIELD not found")
except NameError:
    # Try to find vector store by name
    print("🔄 Attempting to find vector store by name 'single-field-rag-tickets'...")
    try:
        vector_stores = client.vector_stores.list()
        found = False
        for vs in vector_stores.data if hasattr(vector_stores, 'data') else []:
            if hasattr(vs, 'name') and vs.name == 'single-field-rag-tickets':
                VECTOR_STORE_ID_SINGLE_FIELD = vs.id
                VECTOR_STORE_ID = vs.id  # For backward compatibility
                vector_store_id = vs.id
                found = True
                print(f"✅ Found vector store: {vs.id}")
                break
        if not found:
            raise ValueError("Vector store not found")
    except Exception as e:
        print(f"❌ Could not find vector store: {e}")
        print(f"\n💡 Please run 00_data_ingestion.ipynb first!")
        print(f"   It will create both vector stores (single-field and multi-field)")
        raise

# Retrieve vector store information
vs_chroma = client.vector_stores.retrieve(VECTOR_STORE_ID_SINGLE_FIELD)

print(f"\n✅ Vector store loaded!")
print("=" * 60)
print(f"📦 Vector Store Details:")
print(f"   ID: {vs_chroma.id}")
print(f"   Status: {vs_chroma.status}")
if vs_chroma.name:
    print(f"   Name: {vs_chroma.name}")
if vs_chroma.metadata:
    provider = vs_chroma.metadata.get('provider_id', 'N/A')
    print(f"   Provider: {provider}")

# Display document statistics
if hasattr(vs_chroma, 'file_counts') and vs_chroma.file_counts:
    print(f"\n📊 Document Statistics:")
    print(f"   Total files: {vs_chroma.file_counts.total}")
    print(f"   Completed: {vs_chroma.file_counts.completed}")
    print(f"   In progress: {vs_chroma.file_counts.in_progress}")
    print(f"   Failed: {vs_chroma.file_counts.failed}")

print("=" * 60)
print(f"\n💡 Vector store is ready for semantic search!")

**What happened:** We loaded the vector store! ✅

**💡 What is a Vector Store?** It's a database that stores document embeddings. Think of it as a specialized database optimized for finding similar vectors (similar meanings).

**Key point:** The vector store was created and populated in `00_data_ingestion.ipynb`. It already contains indexed tickets ready for semantic search.

**What's next:** Now we can query the vector store to find tickets by meaning, not just keywords!

---

In [ ]:
# Vector store is ready! No need to prepare or index documents here.
# All documents were already indexed in 00_data_ingestion.ipynb

print("✅ Vector store is ready for querying!")
print("💡 Documents were indexed in 00_data_ingestion.ipynb")
print("   - Content field: short_description (problem summary)")
print("   - Metadata: All other ticket fields (for filtering)")

**What happened:** The vector store is ready! ✅

**💡 What's in the Vector Store?**
- **Content:** The `short_description` field (what we'll search)
- **Metadata:** All other ticket fields (for filtering later)

**Why "Simple RAG"?** We're using only one field (`short_description`) for search. This is the simplest approach.

**💡 In notebook 02:** We'll see how combining multiple fields (`short_description` + `content` + `close_notes`) creates richer documents that improve search quality!

**What's next:** Now we can query the vector store to see semantic search in action!

---

In [ ]:
# Display vector store with documents after indexing
print("\n" + "=" * 60)
print("📊 Vector Store Status After Indexing")
print("=" * 60)

# Retrieve updated vector store information
vs_updated = client.vector_stores.retrieve(vs_chroma.id)

print(f"\n📦 Vector Store Details:")
print(f"   ID: {vs_updated.id}")
print(f"   Status: {vs_updated.status}")
if vs_updated.name:
    print(f"   Name: {vs_updated.name}")
if vs_updated.metadata:
    provider = vs_updated.metadata.get("provider_id", "N/A")
    print(f"   Provider: {provider}")

# Display file/document statistics
if hasattr(vs_updated, "file_counts") and vs_updated.file_counts:
    print(f"\n📊 Document Statistics:")
    print(f"   Total files: {vs_updated.file_counts.total}")
    print(f"   Completed: {vs_updated.file_counts.completed}")
    print(f"   In progress: {vs_updated.file_counts.in_progress}")
    print(f"   Failed: {vs_updated.file_counts.failed}")
    print(f"   Cancelled: {vs_updated.file_counts.cancelled}")

# Display storage usage
if hasattr(vs_updated, "usage_bytes") and vs_updated.usage_bytes:
    usage_mb = vs_updated.usage_bytes / (1024 * 1024)
    print(f"\n💾 Storage:")
    print(f"   Usage: {usage_mb:.2f} MB")

# Display timestamps
if hasattr(vs_updated, "created_at") and vs_updated.created_at:
    from datetime import datetime

    created_time = datetime.fromtimestamp(vs_updated.created_at)
    print(f"\n🕒 Timestamps:")
    print(f"   Created: {created_time.strftime('%Y-%m-%d %H:%M:%S')}")
    if hasattr(vs_updated, "last_active_at") and vs_updated.last_active_at:
        last_active = datetime.fromtimestamp(vs_updated.last_active_at)
        print(f"   Last active: {last_active.strftime('%Y-%m-%d %H:%M:%S')}")

# Query the vector store to show sample documents
print(f"\n" + "=" * 60)
print("🔍 Sample Documents in Vector Store")
print("=" * 60)
print("\n💡 Querying vector store to retrieve sample documents...")

try:
    # Query with a general query to get some sample results
    sample_query = "IT support ticket"
    query_result = client.tool_runtime.rag_tool.query(
        content=sample_query,
        vector_db_ids=[str(vs_chroma.id)],
        extra_body={"vector_store_ids": [str(vs_chroma.id)]},
    )

    print(f"\n✅ Vector store is queryable and contains indexed documents!")
    print(f"\n📄 Sample Query Results:")
    print(f"   Query: '{sample_query}'")

    # Extract document information if available
    if hasattr(query_result, "content") and query_result.content:
        content_preview = (
            query_result.content[:300]
            if len(query_result.content) > 300
            else query_result.content
        )
        print(f"\n   Retrieved content preview:")
        print(f"   {content_preview}...")

    print(f"\n💡 The vector store is ready for semantic search!")
    print(f"   You can now query it to find tickets by meaning, not just keywords.")

except Exception as e:
    print(f"\n⚠️  Could not query vector store: {e}")
    print(f"   Vector store may still be processing documents.")

print("=" * 60)

**What happened:** We indexed all documents into ChromaDB! ✅

**🎉 Success!** The tickets are now searchable using semantic similarity. Each ticket description has been:
- ✅ Converted into embeddings (vectors representing meaning)
- ✅ Stored in ChromaDB (ready for semantic search)

**💡 What happened behind the scenes:**
- LlamaStack automatically chunked long descriptions
- Generated embeddings using the embedding model
- Stored them in the vector database

---

### Step 5: Query with RAG - See Semantic Search in Action!

**What we're doing:** Testing our RAG system with a query to see how semantic search works.

**Why:** This is where you'll see the power of RAG - finding relevant tickets even when they use different words. It matches meaning, not just keywords!

**Ready to see semantic search in action?** Let's query our indexed tickets:

**What happened:** We indexed the tickets successfully! ✅

**What's next:** Now let's query the system to see semantic search in action. Watch how it finds tickets with similar meaning, even if they use different words!

---

In [ ]:
# Test query
query = "What was the root cause and resolution for application crashes related to memory issues?"

print("🔍 Querying RAG System")
print("=" * 60)
cprint(f"\n📝 User Query: {query}", "blue")
print("\n💡 This query will use semantic search to find:")
print("   - Tickets about 'application crashes'")
print("   - Tickets about 'memory issues'")
print("   - Even if they use different words!")
print("\n🔄 Searching vector store...")

# Step 1: RAG retrieval - find relevant document chunks
rag_response = client.tool_runtime.rag_tool.query(
    content=query,
    vector_db_ids=[str(vs_chroma.id)],
    extra_body={"vector_store_ids": [str(vs_chroma.id)]},
)

print(f"✅ Retrieved relevant context from vector store")
print(f"\n📄 Retrieved Context (first 500 chars):")
print("=" * 60)
print(rag_response.content[:500] + "..." if len(rag_response.content) > 500 else rag_response.content)
print("\n💡 Notice: The system found relevant tickets using semantic similarity!")

# Step 2: Construct extended prompt with retrieved context
messages = [{"role": "system", "content": "You are a helpful IT support assistant."}]
extended_prompt = f"Please answer the given query using the context below.\n\nCONTEXT:\n{rag_response.content}\n\nQUERY:\n{query}"
messages.append({"role": "user", "content": extended_prompt})

# Step 3: Generate answer using LLM
print("\n🔄 Generating answer with LLM...")
response = client.chat.completions.create(
    messages=messages,
    model=model,
    stream=stream,
    max_tokens=max_tokens,
    temperature=temperature,
)

print("\n✅ Answer:")
print("=" * 60)
if stream:
    for chunk in response:
        if chunk.choices and chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
    print()
else:
    print(response.choices[0].message.content)

**What happened:** We used RAG to find relevant tickets and generate an answer! The system matched meaning, not just keywords.

---

## 🎓 Key Takeaway

RAG (Retrieval-Augmented Generation) enables semantic search - finding documents by meaning, not just exact keyword matches. The data was indexed in `00_data_ingestion.ipynb`, and now we can search for similar incidents even when they use different words. This is the foundation for intelligent IT operations search systems.

---

## 🌍 Real-World Connection

**How this applies to IT Operations:**

The same RAG approach can be used for:

- **Incident Resolution:** "Find similar incidents to this one" → Get past solutions
- **Knowledge Base Search:** Search through documentation and runbooks using natural language
- **Pattern Recognition:** Identify recurring problems across incidents
- **Root Cause Analysis:** Find incidents with similar symptoms to learn from past diagnostics

**The pattern is the same:** Index historical data (in `00_data_ingestion.ipynb`) → Query semantically → Retrieve relevant context → Use context to answer questions or suggest solutions.

---

## ✨ Your Turn

**Try this:** Modify the query to search for different types of problems. For example:
- "How do I fix database connection errors?"
- "What causes slow application performance?"
- "Find tickets about network issues"

Notice how the semantic search finds relevant tickets even with different wording!

---

## 🎉 You Did It!

You've queried your first RAG system! You learned how to use semantic search to find IT tickets by meaning, not just keywords. 

**What you learned:**
- ✅ How to connect to a pre-indexed vector store
- ✅ How to query using semantic similarity
- ✅ How RAG retrieves relevant context and generates answers

**Next Steps:**
1. **For multi-field RAG**: Run `00_data_ingestion.ipynb` with `INGESTION_MODE = "multi-field"`, then use `02_advanced_rag_with_multiple_fields.ipynb`
2. **To re-index data**: Use `00_data_ingestion.ipynb` anytime you need to update the vector store

---

## 📚 Want to Go Deeper?

<details>
<summary>📖 Additional Resources (Click to expand)</summary>

- [LlamaStack Documentation](https://github.com/llamastack/llamastack) - RAG and vector store capabilities
- [ChromaDB Documentation](https://www.trychroma.com/) - Vector database used by LlamaStack
- [RAG Best Practices](https://www.pinecone.io/learn/retrieval-augmented-generation/) - RAG techniques and patterns
</details>